# 1.5 「ファインチューニング」で精度向上を実現する方法

- 本ファイルでは、学習済みのVGGモデルを使用し、ファインチューニングでアリとハチの画像を分類するモデルを学習します



# 学習目標

1.	PyTorchでGPUを使用する実装コードを書けるようになる
2.	最適化手法の設定において、層ごとに異なる学習率を設定したファインチューニングを実装できるようになる
3.	学習したネットワークを保存・ロードできるようになる



# 事前準備

- 1.4節で解説したAWS EC2 のGPUインスタンスを使用します


In [5]:
import torch
torch.cuda.is_available()

True

In [6]:
%load_ext autoreload
%autoreload 2

In [8]:
%load_ext blackcellmagic

In [9]:
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import models

from tqdm import tqdm

In [10]:
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [11]:
import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# DatasetとDataLoaderを作成

In [12]:
from pytorch_advanced.c01_vgg.transfer_learning import ImageTransform, make_datapath_list, HymenopteraDataset

In [13]:
train_list = make_datapath_list("train")
val_list = make_datapath_list("val")

INFO:pytorch_advanced.c01_vgg.transfer_learning:Load 243 train files
INFO:pytorch_advanced.c01_vgg.transfer_learning:Load 153 val files


In [14]:
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

In [15]:
train_dataset = HymenopteraDataset(
    file_list=train_list,
    transform=ImageTransform(size, mean, std),
    phase="train",
)

val_dataset = HymenopteraDataset(
    file_list = val_list,
    transform = ImageTransform(size, mean, std),
    phase="val",
)

In [16]:
batch_size = 32
train_dataloder = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
)

val_dataloder = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=True,
)

dataloaders_dict = {
    "train": train_dataloder,
    "val": val_dataloder,
}

# ネットワークモデルの作成

In [17]:
pretrained = True
net = models.vgg16(pretrained=pretrained)

net.classifier[6] = nn.Linear(in_features=4096, out_features=2)

net.train()
print("In train mode")

In train mode


# 損失関数を定義

In [18]:
criterion = nn.CrossEntropyLoss()

# 最適化手法を設定

In [19]:
params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

In [20]:
update_param_names_1 = ["features"]
update_param_names_2 = [f"classifier.{n}.{wb}" for n in [0, 3] for wb in ["weight", "bias"]]
update_param_names_3 = [f"classifier.{n}.{wb}" for n in [6] for wb in ["weight", "bias"]]

In [21]:
for name, param in net.named_parameters():
    if update_param_names_1[0] in name:
        param.requires_grad = True
        params_to_update_1.append(param)
        logger.info(f"In params_to_update_1: {name}")
    elif name in update_param_names_2:
        param.requires_grad = True
        params_to_update_2.append(param)
        logger.info(f"In params_to_update_2: {name}")
    elif name in update_param_names_3:
        param.requires_grad = True
        params_to_update_3.append(param)
        logger.info(f"In params_to_update_3: {name}")
    else:
        param.requires_grad = False
        logger.info(f"No grads: {name}")

INFO:root:In params_to_update_1: features.0.weight
INFO:root:In params_to_update_1: features.0.bias
INFO:root:In params_to_update_1: features.2.weight
INFO:root:In params_to_update_1: features.2.bias
INFO:root:In params_to_update_1: features.5.weight
INFO:root:In params_to_update_1: features.5.bias
INFO:root:In params_to_update_1: features.7.weight
INFO:root:In params_to_update_1: features.7.bias
INFO:root:In params_to_update_1: features.10.weight
INFO:root:In params_to_update_1: features.10.bias
INFO:root:In params_to_update_1: features.12.weight
INFO:root:In params_to_update_1: features.12.bias
INFO:root:In params_to_update_1: features.14.weight
INFO:root:In params_to_update_1: features.14.bias
INFO:root:In params_to_update_1: features.17.weight
INFO:root:In params_to_update_1: features.17.bias
INFO:root:In params_to_update_1: features.19.weight
INFO:root:In params_to_update_1: features.19.bias
INFO:root:In params_to_update_1: features.21.weight
INFO:root:In params_to_update_1: featu

In [22]:
optimizer = optim.SGD([
    {'params': params_to_update_1, 'lr': 1e-4},
    {'params': params_to_update_2, 'lr': 5e-4},
    {'params': params_to_update_3, 'lr': 1e-3},
], momentum=0.9)

In [25]:
type(net)

torchvision.models.vgg.VGG

In [26]:
!pip list | grep pytorch

pytorch-advanced              0.1.1


# 学習・検証を実施

In [27]:
from pytorch_advanced.c01_vgg.fine_tuning import train_model

In [29]:
# 学習・検証を実行する
num_epochs=2
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

INFO:pytorch_advanced.c01_vgg.fine_tuning:Using cuda:0
INFO:pytorch_advanced.c01_vgg.fine_tuning:Epoch 1/2
INFO:pytorch_advanced.c01_vgg.fine_tuning:------------------------------
100%|██████████| 5/5 [00:03<00:00,  1.53it/s]
INFO:pytorch_advanced.c01_vgg.fine_tuning:val Loss: 0.7704 Acc: 0.4444
INFO:pytorch_advanced.c01_vgg.fine_tuning:Epoch 2/2
INFO:pytorch_advanced.c01_vgg.fine_tuning:------------------------------
100%|██████████| 8/8 [00:10<00:00,  1.25s/it]
INFO:pytorch_advanced.c01_vgg.fine_tuning:train Loss: 0.4861 Acc: 0.7284
100%|██████████| 5/5 [00:01<00:00,  2.53it/s]
INFO:pytorch_advanced.c01_vgg.fine_tuning:val Loss: 0.1719 Acc: 0.9542


# 学習したネットワークを保存・ロード

In [30]:
# PyTorchのネットワークパラメータの保存
save_path = './weights_fine_tuning.pth'
torch.save(net.state_dict(), save_path)

In [31]:
# PyTorchのネットワークパラメータのロード
load_path = './weights_fine_tuning.pth'
load_weights = torch.load(load_path)
net.load_state_dict(load_weights)

# GPU上で保存された重みをCPU上でロードする場合
load_weights = torch.load(load_path, map_location={'cuda:0': 'cpu'})
net.load_state_dict(load_weights)


<All keys matched successfully>

以上